# Retrieval Pipeline Orchestration + Reranking

**Goal**
Build a hybrid retrieval system that:

1. Pulls candidate results from your Qdrant vector store (semantic retrieval).

2. Optionally augments those with graph-based or metadata filters (hierarchical or relational layer).

3. Applies cross-encoder reranking to refine relevance before passing context to the LLM.

# 1: Setup & Connections

* Import required libraries.
* Load/connect to Qdrant (vector DB) and Memgraph (graph DB).
* Load the embedding model (for encoding queries) and the cross-encoder reranker (for re-scoring * candidate documents).
* Define some configuration variables (collection name, top_k defaults).
* Print short status messages so you know everything connected/loaded correctly.

In [1]:
# Before running this notebook, ensure you have run Notebook 01 and Notebook 02 to set up the databases and ingest data.
# Also, make sure your Qdrant and Memgraph services are running.
# Run docker commands from root project @ terminal: 
# docker-compose up -d
# Or, if running individually:
# docker run -p 6333:6333 qdrant/qdrant
# docker run -it -p 7687:7687 -p 3000:3000 memgraph/memgraph-platform

#  Standard libs 
import os
import json
from typing import List, Dict, Any
from pprint import pprint

#  Utilities 
from tqdm import tqdm

#  Qdrant client (vector DB) 
# pip package: qdrant-client
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

#  Memgraph client (graph DB) 
# pip package: gqlalchemy
from gqlalchemy import Memgraph

#  Sentence Transformers (embeddings + cross-encoder) 
# pip package: sentence-transformers
from sentence_transformers import SentenceTransformer, CrossEncoder

#  Configuration: modify if your services run on different hosts/ports 
QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6333")
MEMGRAPH_HOST = os.getenv("MEMGRAPH_HOST", "localhost")
MEMGRAPH_PORT = int(os.getenv("MEMGRAPH_PORT", 7687))

# Name of the Qdrant collection we created in Notebook 02.
# Change needed later to allow for multiple collections
COLLECTION_NAME = os.getenv("QDRANT_COLLECTION", "enterprise_docs")

# How many candidates to fetch from each retriever before reranking
DEFAULT_TOP_K = 50        # number of candidates from vector search
FINAL_TOP_K = 5           # final number of results to return to user after rerank

#  Connect to Qdrant 
try:
    qdrant_client = QdrantClient(url=QDRANT_URL)
    # A small call to verify the collection exists (raises if unreachable)
    collections = qdrant_client.get_collections()
    print(f"✅ Connected to Qdrant at {QDRANT_URL}. Collections: {[c.name for c in collections.collections]}")
except Exception as e:
    qdrant_client = None
    print("❌ Could not connect to Qdrant:", e)

#  Connect to Memgraph 
try:
    memgraph = Memgraph(host=MEMGRAPH_HOST, port=MEMGRAPH_PORT)
    # Simple test query to verify connection (returns a single row)
    test = memgraph.execute_and_fetch("RETURN 1 AS ok")
    _ = list(test)  # consume generator
    print(f"✅ Connected to Memgraph at {MEMGRAPH_HOST}:{MEMGRAPH_PORT}")
except Exception as e:
    memgraph = None
    print("❌ Could not connect to Memgraph:", e)

#  Load embedding model for queries 
# Use the same model family you used to embed documents in Notebook 02.
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL", "sentence-transformers/all-MiniLM-L6-v2")
try:
    embedder = SentenceTransformer(EMBEDDING_MODEL_NAME)
    print(f"✅ Loaded embedding model: {EMBEDDING_MODEL_NAME}")
except Exception as e:
    embedder = None
    print("❌ Could not load embedding model:", e)

# Coross-encoder for reranking choices depend largely on available resources and implementation needs.
# Load cross-encoder reranker (can be large) 
# Choose a comparatively small reranker for local usage if resources are limited:
# e.g., "cross-encoder/ms-marco-MiniLM-L-6-v2"
RERANKER_MODEL_NAME = os.getenv("RERANKER_MODEL", "cross-encoder/ms-marco-MiniLM-L-6-v2")
try:
    reranker = CrossEncoder(RERANKER_MODEL_NAME)
    print(f"✅ Loaded reranker model: {RERANKER_MODEL_NAME}")
except Exception as e:
    reranker = None
    print("⚠️ Could not load reranker model (this is optional). Error:", e)

#  Quick summary of loaded components 
print("\n --- Summary --- ")
print(f"Qdrant client: {'OK' if qdrant_client else 'MISSING'}")
print(f"Memgraph client: {'OK' if memgraph else 'MISSING'}")
print(f"Embedder: {'OK' if embedder else 'MISSING'} ({EMBEDDING_MODEL_NAME})")
print(f"Reranker: {'OK' if reranker else 'MISSING'} ({RERANKER_MODEL_NAME})")
print(f"Qdrant collection (to use): {COLLECTION_NAME}")
print(f"Retrieval settings: DEFAULT_TOP_K={DEFAULT_TOP_K}, FINAL_TOP_K={FINAL_TOP_K}")


✅ Connected to Qdrant at http://localhost:6333. Collections: ['enterprise_docs']
✅ Connected to Memgraph at localhost:7687
✅ Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ Loaded reranker model: cross-encoder/ms-marco-MiniLM-L-6-v2

 --- Summary --- 
Qdrant client: OK
Memgraph client: OK
Embedder: OK (sentence-transformers/all-MiniLM-L6-v2)
Reranker: OK (cross-encoder/ms-marco-MiniLM-L-6-v2)
Qdrant collection (to use): enterprise_docs
Retrieval settings: DEFAULT_TOP_K=50, FINAL_TOP_K=5


# 2 Semantic Retrieval from Qdrant

Here we will:
1. Take a natural language query.
2. Encode it into a vector using your embedding model.
3. Search for the most semantically similar chunks (documents) in Qdrant.
4. Return and display the top K results, with metadata for inspection.

In [2]:
# Define a function to search Qdrant using vector similarity
def semantic_retrieval(
    query: str,
    top_k: int = DEFAULT_TOP_K,
    collection_name: str = COLLECTION_NAME
) -> List[Dict[str, Any]]:
    """
    Retrieve the top_k most similar chunks for a given query using cosine similarity.
    Returns: List of dictionaries containing metadata + scores.
    """
    # Encode the input query into a vector
    query_vector = embedder.encode(query).tolist()

    # Search the Qdrant collection
    # We use 'search' to get the most similar documents by cosine similarity
    # 'with_payload=True' ensures we also retrieve metadata (like source, page, etc.)
    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k,
        with_payload=True,
    )

    # Parse and format results for easier handling
    formatted_results = []
    for r in results:
        payload = r.payload or {}
        formatted_results.append({
            "id": r.id,
            "score": r.score,  # similarity score
            "text": payload.get("text", ""),  # the actual content chunk
            "source": payload.get("source", "unknown"),
            "page": payload.get("page", None),
            "metadata": payload,
        })

    return formatted_results

# ===========================================
# Example Query — try one from your dataset
# ===========================================
# Replace this with a question relevant to your data.
sample_query = "What are the key features of RIS technology?"

# Run the retrieval function
retrieved_docs = semantic_retrieval(sample_query, top_k=5)

# Display retrieved documents
print(f"\n🔎 Top {len(retrieved_docs)} results for query: '{sample_query}'\n")
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"Result {i}: (score={doc['score']:.4f})")
    print(f"Source: {doc['source']} | Page: {doc.get('page')}")
    print(f"Snippet: {doc['text'][:300]}...\n")


🔎 Top 5 results for query: 'What are the key features of RIS technology?'

Result 1: (score=0.3908)
Source: ../data/test.pdf | Page: None
Snippet: intent-based management should be deployed to allow human-network interaction [3]. To this end, human operators use high-level declarative languages to define system operational goals in the form of ”intents, which are then translated to lower-level instructions (action plans and settings) using int...

Result 2: (score=0.3607)
Source: ../data/test.pdf | Page: None
Snippet: anyone? The essence of the Io S, coined by Ericsson [4], is built on delivering multisensory experiences over the networks. This UC allows humans to have digital sensory immersive experiences that replicate or even augment what we experience in the physical world through visual, auditory, haptic, ol...

Result 3: (score=0.3191)
Source: ../data/test.pdf | Page: None
Snippet: be needed to facilitate automation, where most of the development and deployment of distributed ap

/tmp/ipykernel_10448/3143761866.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


# 3 Hybrid Retrieval (Vector + Graph)
This next cell implements a hybrid retrieval layer that merges: 
1. **Semantic relevance**
2. **Graph-based context**

This hybrid approach improves entreprise knowledge retrieval by connecting related entities instead of relying exclusively on text similarity. 

In [3]:
def graph_retrieval(
    query: str,
    memgraph_conn: Memgraph,
    limit: int = 10
) -> List[Dict[str, Any]]:
    """
    Retrieve additional context from Memgraph based on related nodes or edges.
    You can customize the Cypher query for your use case.
    Example: retrieve nodes whose 'name' or 'description' matches the query.
    """
    if memgraph_conn is None:
        print(" Memgraph not connected — skipping graph retrieval.")
        return []

    # Simple Cypher example 
    # This assumes your graph has nodes with a 'name' or 'text' property.
    # The cypher_query can be modified to suit the context of Memgraph schema.
    cypher_query = f"""
    MATCH (n)
    WHERE toLower(n.name) CONTAINS toLower("{query}")
       OR toLower(n.text) CONTAINS toLower("{query}")
    RETURN n.name AS name, n.text AS text, labels(n) AS labels
    LIMIT {limit}
    """

    try:
        results = list(memgraph_conn.execute_and_fetch(cypher_query))
        # Normalize output to consistent dict structure
        return [
            {
                "source": "memgraph",
                "text": r.get("text", r.get("name", "")),
                "labels": r.get("labels", []),
                "score": 0.5,  # neutral base score; reranker will adjust later
            }
            for r in results
        ]
    except Exception as e:
        print("Graph retrieval failed:", e)
        return []


def hybrid_retrieval_pipeline(query: str, top_k_semantic: int = DEFAULT_TOP_K) -> List[Dict[str, Any]]:
    """
    1. Retrieve semantic candidates from Qdrant.
    2. Retrieve graph-based context from Memgraph.
    3. Merge, deduplicate, and prepare for reranking.
    """
    #  Step 1: Vector-based retrieval
    semantic_results = semantic_retrieval(query, top_k=top_k_semantic)

    #  Step 2: Graph-based retrieval 
    graph_results = graph_retrieval(query, memgraph_conn=memgraph)

    #  Step 3: Merge results 
    # Simply concatenate lists for now; deduplication can be added if overlap exists
    combined_results = semantic_results + graph_results

    print(f"Retrieved {len(semantic_results)} semantic + {len(graph_results)} graph results.")
    return combined_results


# Example hybrid query
# ===========================================
sample_query = "What does 6G offer?"

hybrid_results = hybrid_retrieval_pipeline(sample_query, top_k_semantic=10)

# Preview results
print(f"\n Combined results for query: '{sample_query}'\n")
for i, doc in enumerate(hybrid_results[:10], start=1):
    print(f"Result {i}: [Source: {doc.get('source', 'unknown')}] Score: {doc.get('score', 'N/A')}")
    print(f"Snippet: {doc.get('text', '')[:200]}...\n")

Retrieved 10 semantic + 0 graph results.

 Combined results for query: 'What does 6G offer?'

Result 1: [Source: ../data/test.pdf] Score: 0.46595055
Snippet: The Journey A Digital and Societal Lina Mohjazi, Bassant Selim, Mallik Abstract—While the fifth generation (5 G) is bringing an innovative fabric of breakthrough technologies, enabling smart factories...

Result 2: [Source: ../data/test.pdf] Score: 0.4221406
Snippet: anticipated 6 G promise into reality. REFERENCES [1] M. Al-Quraan, L. Mohjazi, L. Bariah, A. Centeno, A. Zoha, K. Arshad, K. Assaleh, S. Muhaidat, M. Debbah, and M. A. Imran, “Edge-native intelligence...

Result 3: [Source: ../data/test.pdf] Score: 0.4129762
Snippet: M.A.Imranarewiththe James Watt Schoolof Engineering, University of Glasgow, Glasgow, G 12 8 QQ, UK. (e-mail:{Lina. Mohjazi, Muhammad.Imran}@glasgow.ac.uk). B. Selim is with E´cole de Technologie Supe´...

Result 4: [Source: ../data/test.pdf] Score: 0.32941002
Snippet: e.g., mobile user equipment connected

/tmp/ipykernel_10448/3143761866.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


# 4 Semantic Reranking

(after hybrid retrieval)

This cell takes everything retrieved in Cell 3, and reorders the results using a cross-encoder, which computes true, pairwise relevance between the query and each candidate chunk.

In [4]:
# Expected Inputs:
# - `query` (string): the user's search query
# - `hybrid_results` (list): results returned from Cell 3 (vector + graph)
#
# Output:
# - In-order, relevance-boosted results using a cross-encoder
# -------------------------------------------------------------

from sentence_transformers import CrossEncoder

# Load a lightweight, fast, open-source reranker
RERANKER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANKER_MODEL)

def rerank_results(query: str, results: list, top_n: int = 5):
    """
    Apply cross-encoder reranking to hybrid-retrieved results.
    
    The cross encoder evaluates each (query, passage) pair directly.
    This usually gives 20–40% better top-1 accuracy than embeddings alone.
    """
    
    # Build (query, text) pairs for model inference
    pairs = [(query, r["text"]) for r in results]
    
    # Predict semantic similarity scores
    scores = reranker.predict(pairs)
    
    # Attach new scores
    for r, s in zip(results, scores):
        r["rerank_score"] = float(s)
    
    # Sort descending by score
    reranked = sorted(results, key=lambda x: x["rerank_score"], reverse=True)
    
    return reranked[:top_n]


# ---------------------------
# 🔍 Try it out
# ---------------------------
query = "What does 6G offer?"
reranked = rerank_results(query, hybrid_results, top_n=5)

# Pretty print
for i, item in enumerate(reranked, 1):
    print(f"Reranked {i} — Score: {item['rerank_score']:.4f}")
    print(f"Source: {item['source']}")
    print(f"Text snippet: {item['text'][:250]}...")
    print("-" * 80)

Reranked 1 — Score: 0.0978
Source: ../data/test.pdf
Text snippet: The Journey A Digital and Societal Lina Mohjazi, Bassant Selim, Mallik Abstract—While the fifth generation (5 G) is bringing an innovative fabric of breakthrough technologies, enabling smart factories, cities, and Internet-of-Things (Io T), the unpre...
--------------------------------------------------------------------------------
Reranked 2 — Score: -1.5797
Source: ../data/test.pdf
Text snippet: anticipated 6 G promise into reality. REFERENCES [1] M. Al-Quraan, L. Mohjazi, L. Bariah, A. Centeno, A. Zoha, K. Arshad, K. Assaleh, S. Muhaidat, M. Debbah, and M. A. Imran, “Edge-native intelligence for 6 G communications driven by federated learni...
--------------------------------------------------------------------------------
Reranked 3 — Score: -2.8368
Source: ../data/test.pdf
Text snippet: 6 G Architectural Pillars. which can be of diverse and new generations of modalities, leading the data processing to be highly dis

In [5]:
# -------------------------------------------------------------
# 📦 Cell 5: Build Final Context + Citation Packaging
# -------------------------------------------------------------
# Input: `reranked` → output from Cell 4
# Output: `context_for_llm` → dictionary containing:
#   - merged_context (clean, deduped text)
#   - citations list (source + chunk_id)
#   - raw_chunks (original chunk text)
# -------------------------------------------------------------

import re

def clean_and_deduplicate(chunks: list) -> list:
    """
    Deduplicate chunks by removing near-identical text blocks.
    Basic but effective dedup for RAG pipelines.
    """
    seen = set()
    unique_chunks = []
    
    for text in chunks:
        key = re.sub(r"\s+", " ", text.strip().lower())
        if key not in seen:
            seen.add(key)
            unique_chunks.append(text)
    
    return unique_chunks


def assemble_context(reranked_results: list, max_chars: int = 8000):
    """
    Merge reranked chunks into a final LLM-ready context block.
    Includes citation metadata.
    """
    # Extract text chunks
    raw_chunks = [item["text"] for item in reranked_results]
    
    # Deduplicate
    unique_chunks = clean_and_deduplicate(raw_chunks)
    
    # Build the merged context (bounded by max_chars)
    merged_text = ""
    citations = []
    
    for item, text in zip(reranked_results, unique_chunks):
        
        if len(merged_text) + len(text) > max_chars:
            break
        
        merged_text += text.strip() + "\n\n---\n\n"
        
        citations.append({
            "source": item["source"],
            "chunk_preview": text[:120] + "...",
            "rerank_score": item.get("rerank_score", None)
        })
    
    return {
        "merged_context": merged_text.strip(),
        "citations": citations,
        "raw_chunks": unique_chunks
    }


# ---------------------------
# 🔍 Try it with your reranked results
# ---------------------------
context_for_llm = assemble_context(reranked, max_chars=8000)

print("📦 Final Context Block Ready\n")
print(context_for_llm["merged_context"][:800])  # preview first 800 chars
print("\n🔗 Citations:")
for c in context_for_llm["citations"]:
    print(f"- {c['source']} | {c['chunk_preview']}")

📦 Final Context Block Ready

The Journey A Digital and Societal Lina Mohjazi, Bassant Selim, Mallik Abstract—While the fifth generation (5 G) is bringing an innovative fabric of breakthrough technologies, enabling smart factories, cities, and Internet-of-Things (Io T), the unprecedented strain on communication networks put by these applications, in terms of highly cognitive, agile architectures and the support of massive connectivity, energy efficiency, and extreme ultralow latency, is pushing 5 G to their limits. As such, the focus of academic and industrial efforts has shifted toward beyond 5 G (B 5 G) and the conceptualization of sixth generation (6 G) systems. This article discusses four main digital and societal use cases (UCs) that will drive the need to reconcile a new breed of network requirements. Based on th

🔗 Citations:
- ../data/test.pdf | The Journey A Digital and Societal Lina Mohjazi, Bassant Selim, Mallik Abstract—While the fifth generation (5 G) is brin...
- ../data/t

In [6]:
print("context_for_llm exists:", 'context_for_llm' in locals())

if 'context_for_llm' in locals():
    print(context_for_llm.keys())

context_for_llm exists: True
dict_keys(['merged_context', 'citations', 'raw_chunks'])


# LLM Answering Pipeline
This section is responsible for:

* Running a local, open-source LLM
* Constructing the final prompt (query + context)
* Generating an answer
* Attaching citations
* Returning a polished response

Implementing open-source software, we will use:

Ollama (best, simplest, FREE local LLM runner)

Model: llama3 (this can changed eventually in order to improve)

In [7]:
# Imports + Global Settings

import json
import os
from datetime import datetime
from typing import Dict

# Optional: pretty printing
import textwrap

# Load Ollama local LLM client (try the official package first, fall back to HTTP)
# Preferred: `pip install ollama`
llm = None
try:
    from ollama import Client as OllamaClient
    llm = OllamaClient(host="http://localhost:11434")
    print("Local LLM client ready (ollama package).")
except Exception as e:
    # Fallback: lightweight HTTP client using requests
    try:
        import requests
        from requests.exceptions import HTTPError, RequestException

        class SimpleOllamaClient:
            def __init__(self, host: str = "http://localhost:11434"):
                self.host = host.rstrip("/")

            def _normalize(self, data):
                # Normalize common response shapes to a consistent dict
                # Ollama's primary shape: {"response": "...", "model": "...", ...}
                if isinstance(data, dict):
                    # Check for Ollama's native "response" key first
                    if "response" in data:
                        return {"message": {"content": data["response"]}}
                    # Check for standard message.content shape
                    if "message" in data and isinstance(data["message"], dict) and "content" in data["message"]:
                        return data
                    # Check for simple text key
                    if "text" in data:
                        return {"message": {"content": data["text"]}}
                    # Check for choices array (OpenAI-like)
                    if "choices" in data and isinstance(data["choices"], list) and len(data["choices"]) > 0:
                        first = data["choices"][0]
                        if isinstance(first, dict) and ("text" in first or "message" in first):
                            text = first.get("text") or (first.get("message") if isinstance(first.get("message"), str) else (first.get("message", {}).get("content") if isinstance(first.get("message"), dict) else None))
                            return {"message": {"content": text}}
                # Fallback: stringify
                return {"message": {"content": str(data)}}

            def chat(self, model: str, messages: list):
                """Send a consolidated prompt to Ollama HTTP API and return a normalized response.

                Returns a dict containing `{"message": {"content": <str>}}` on success.
                """
                prompt_text = "\n".join([m.get("content", "") for m in messages if m.get("role") == "user"]).strip()

                url = f"{self.host}/api/generate"
                payload = {"model": model, "prompt": prompt_text, "stream": False}

                try:
                    resp = requests.post(url, json=payload, timeout=60)
                    resp.raise_for_status()
                except HTTPError as he:
                    status = getattr(he.response, "status_code", None)
                    if status == 404:
                        raise RuntimeError(
                            f"Ollama HTTP endpoint not found (404). Is the Ollama daemon running at {self.host}?"
                        ) from he
                    raise
                except RequestException as re:
                    raise RuntimeError(
                        f"Failed to reach Ollama at {self.host}: {re}. Ensure the daemon is running and reachable."
                    ) from re

                try:
                    data = resp.json()
                except ValueError:
                    # Non-JSON response
                    return {"message": {"content": resp.text}}

                return self._normalize(data)

        llm = SimpleOllamaClient(host="http://localhost:11434")
        print("Using fallback HTTP Ollama client (requests).")
    except Exception as e2:
        print("Could not import `ollama` package or use HTTP fallback.")
        print("To enable the local LLM client, either:")
        print("  1) pip install ollama")
        print("  2) ensure Ollama daemon is running at http://localhost:11434 and install requests (`pip install requests`)")
        print("Falling back to a stub llm object that raises if used.")

        class _StubLLM:
            def chat(self, *args, **kwargs):
                raise RuntimeError("No Ollama client available. Install `ollama` or `requests` and start the Ollama daemon.")

        llm = _StubLLM()


Using fallback HTTP Ollama client (requests).


In [8]:
# ---------------------------------------------------------------
# 📘 Notebook 04 — Cell 2
# Prompt Template + Formatting Function
# ---------------------------------------------------------------

def build_rag_prompt(query: str, context_data: Dict):
    """
    Build the final RAG prompt to send to the local LLM.
    Includes:
    - merged context
    - user question
    - instruction to use citations
    """

    context_block = context_data["merged_context"]

    prompt = f"""
You are a helpful assistant answering questions using ONLY the context provided.

CONTEXT:
-------
{context_block}
-------

INSTRUCTIONS:
- Use ONLY the information in the context.
- Do NOT hallucinate.
- Cite your sources using this format: [source].
- If the answer is not in the context, say: "The answer is not available in the provided context."

USER QUESTION:
{query}

FINAL ANSWER (with citations):
"""

    return prompt.strip()


In [9]:
# ---------------------------------------------------------------
# 📘 Notebook 04 — Cell 3
# Generate Answer using Local LLM (Ollama)
# ---------------------------------------------------------------

def generate_llm_answer(query: str, context_for_llm: Dict, model: str = None):
    """
    Sends the final prompt to the local LLM and retrieves the response.
    If model is not specified, uses OLLAMA_MODEL env var (default: llama3.1:latest).
    """
    if model is None:
        model = os.getenv("OLLAMA_MODEL", "llama3.1:latest")
    
    prompt = build_rag_prompt(query, context_for_llm)

    try:
        response = llm.chat(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
    except RuntimeError:
        # Re-raise runtime errors from the client (e.g., helpful 404 message)
        raise
    except Exception as e:
        raise RuntimeError(f"LLM call failed: {e}") from e

    # Attempt to extract a text response from a range of possible shapes
    content = None
    if isinstance(response, dict):
        # Check for normalized message.content first (most common after _normalize)
        if "message" in response and isinstance(response["message"], dict) and "content" in response["message"]:
            content = response["message"]["content"]
        # Fallback checks in case _normalize wasn't applied
        elif "response" in response:
            content = response["response"]
        elif "text" in response:
            content = response["text"]
        elif "choices" in response and isinstance(response["choices"], list) and len(response["choices"]) > 0:
            first = response["choices"][0]
            if isinstance(first, dict):
                content = first.get("text") or (first.get("message") if isinstance(first.get("message"), str) else (first.get("message", {}).get("content") if isinstance(first.get("message"), dict) else None))

    if content is None:
        # Fallback: stringify whatever we got
        content = str(response)

    return content


# ---- Test the answering pipeline (only if variables are defined) ----
# If 'context_for_llm' comes from notebook 03, use it. Otherwise, use a sample query.
if 'context_for_llm' in locals():
    # Use the query and context from notebook 03
    sample_answer = generate_llm_answer(query, context_for_llm)
    
    print("\n===== LLM ANSWER =====\n")
    print(sample_answer)
else:
    print("⚠️ Variable 'context_for_llm' not found in kernel.")
    print("Please run Notebook 03 first to generate the retrieval context, then return here.")


===== LLM ANSWER =====

According to the context, 6G offers the unification of experiences across the digital, physical, and human worlds [1], as well as a bouquet of unique expectations that redefine how we live and protect our planet. Additionally, it is expected to support Internet-of-Everything (IoE) services and radical advancements in human-machine interaction technologies [1].


In [10]:
# ---------------------------------------------------------------
# 📘 Notebook 04 — Cell 4
# Final Output Formatting (Answer + Citations)
# ---------------------------------------------------------------

def format_final_output(answer: str, context_for_llm: Dict):
    """
    Returns a clean, structured output including:
    - Answer
    - Citations
    """
    print("\n=======================")
    print("FINAL ANSWER")
    print("=======================\n")
    print(answer)

    print("\n=======================")
    print("CITATIONS")
    print("=======================\n")
    for c in context_for_llm["citations"]:
        print(f"- Source: {c['source']}")
        print(f"  Preview: {c['chunk_preview']}")
        print(f"  Score: {c['rerank_score']:.4f}\n")


# Display final result (only if variables are defined)
if 'sample_answer' in locals() and 'context_for_llm' in locals():
    format_final_output(sample_answer, context_for_llm)
else:
    print("⚠️ Variables 'sample_answer' and/or 'context_for_llm' not found.")
    print("Please run cells above first to generate these variables.")


FINAL ANSWER

According to the context, 6G offers the unification of experiences across the digital, physical, and human worlds [1], as well as a bouquet of unique expectations that redefine how we live and protect our planet. Additionally, it is expected to support Internet-of-Everything (IoE) services and radical advancements in human-machine interaction technologies [1].

CITATIONS

- Source: ../data/test.pdf
  Preview: The Journey A Digital and Societal Lina Mohjazi, Bassant Selim, Mallik Abstract—While the fifth generation (5 G) is brin...
  Score: 0.0978

- Source: ../data/test.pdf
  Preview: anticipated 6 G promise into reality. REFERENCES [1] M. Al-Quraan, L. Mohjazi, L. Bariah, A. Centeno, A. Zoha, K. Arshad...
  Score: -1.5797



In [11]:
# ---- Ollama HTTP diagnostic (run this cell to test your local Ollama docker)
import os
print("--- Ollama HTTP Diagnostic ---")
host = os.getenv('OLLAMA_HOST', 'http://localhost:11434')
model = os.getenv('OLLAMA_MODEL', 'llama3.1:latest')
print(f'Using host={host} model={model}')

try:
    import requests
    print('requests version:', requests.__version__)
except Exception as e:
    print('requests not available in this kernel:', e)

# Quick GET checks for common endpoints
for path in ['', '/api', '/api/models', '/api/generate']:
    url = host.rstrip('/') + path
    try:
        r = requests.get(url, timeout=4)
        print(f'GET {path} ->', r.status_code)
    except Exception as e:
        print(f'GET {path} error:', e)

# Try a small generate POST to /api/generate
payload = {'model': model, 'prompt': 'Say hello and identify the model used.', 'stream': False}
try:
    gen_url = host.rstrip('/') + '/api/generate'
    r = requests.post(gen_url, json=payload, timeout=20)
    print('POST /api/generate ->', r.status_code)
    text = r.text
    print('Response (truncated):', text[:800])
    try:
        j = r.json()
        print('JSON keys:', list(j.keys()))
        # If llm object exists with _normalize, try to normalize and show result
        if 'llm' in globals() and hasattr(llm, '_normalize'):
            try:
                print('Normalized sample:', llm._normalize(j))
            except Exception as e:
                print('Normalization error:', e)
        else:
            # Try simple normalization heuristics
            if isinstance(j, dict):
                if 'message' in j:
                    print('message:', j.get('message'))
                elif 'text' in j:
                    print('text:', j.get('text'))
                elif 'choices' in j and isinstance(j['choices'], list) and len(j['choices'])>0:
                    print('choices[0]:', j['choices'][0])
    except Exception:
        pass
except Exception as e:
    print('POST /api/generate error:', e)

print('Diagnostic complete. If you see 404 from POST, verify the Ollama daemon and model name (try OLLAMA_MODEL=llama3.1:latest).')


--- Ollama HTTP Diagnostic ---
Using host=http://localhost:11434 model=llama3.1:latest
requests version: 2.32.5
GET  -> 200
GET /api -> 404
GET /api/models -> 404
GET /api/generate -> 405
POST /api/generate -> 200
Response (truncated): {"model":"llama3.1:latest","created_at":"2025-11-15T20:04:34.737354869Z","response":"Hello! I'm a large language model, specifically a variant of the BERT (Bidirectional Encoder Representations from Transformers) architecture, fine-tuned on a variety of tasks to generate human-like responses.\n\nMy specific model is based on the T5 (Text-To-Text Transfer Transformer) framework, which allows me to understand and respond to a wide range of natural language inputs. This combination of BERT and T5 enables me to provide helpful and accurate answers to your questions!","done":true,"done_reason":"stop","context":[128006,882,128007,271,46864,24748,323,10765,279,1646,1511,13,128009,128006,78191,128007,271,9906,0,358,2846,264,3544,4221,1646,11,11951,264,11678,315,